In [1]:
import numpy as np
import pandas as pd
import json
from shapely.geometry import shape, Point



,INCIDENT_TYPE_DESCRIPTION,FROMDATE,WEAPONTYPE,Shooting,DOMESTIC,Year,Month,DAY_WEEK,Location
0,RESIDENTIAL BURGLARY,07/08/2012 06:00:00 AM,Other,No,No,2012,7,Sunday,"(42.34638135, -71.10379454)"
1,AGGRAVATED ASSAULT,07/08/2012 06:03:00 AM,Firearm,Yes,No,2012,7,Sunday,"(42.31684135, -71.07458456)"
2,ROBBERY,07/08/2012 06:26:00 AM,Firearm,No,No,2012,7,Sunday,"(42.34284135, -71.09698955)"
3,COMMERCIAL BURGLARY,07/08/2012 06:56:00 AM,Other,No,No,2012,7,Sunday,"(42.3164411, -71.06582908)"
4,ROBBERY,07/08/2012 07:15:00 AM,Firearm,No,No,2012,7,Sunday,"(42.27051636, -71.11989955)"
5,ROBBERY,07/08/2012 07:32:00 AM,Firearm,Yes,No,2012,7,Sunday,"(42.31328183, -71.0530059)"
6,ROBBERY,07/08/2012 07:50:00 AM,Firearm,No,No,2012,7,Sunday,"(42.32425136, -71.08620956)"
7,SIMPLE ASSAULT,07/08/2012 07:50:00 AM,Unarmed,No,No,2012,7,Sunday,"(42.34924634, -71.06378456)"
8,MedAssist,07/08/2012 07:53:00 AM,Unarmed,No,No,2012,7,Sunday,"(42.35174635, -71.16590953)"
9,MedAssist,07/08/2012 08:05:00 AM,Unarmed,No,No,2012,7,Sunday,"(42.25938275, -71.11729354)"


In [169]:
# somerville happiness data
happy = pd.read_csv('somerville_happy.csv')
happy.columns = [
            'ID', 'Year', 'Current Happy',
           'Overall Life Satisfaction',
           'Somerville Satisfaction',
           'Individual Similarity To Acquaitances 2011',
           'Rely on advice or self for decision making 2011',
           'Neighborhood satisfaction',
           'Proud to be Somerville resident 2015',
           'City Services Availability Rating 2015',
           'Availability of affordable housing 2011',
           'Cost of Housing Rating',
           'Rate cost of public schools 2011',
           'Rate overall cost of public schools 2011',
           'Rate beauty or physical setting 2011',
           'Rate beauty or physical setting 2013',
           'Rate effectiveness of local police 2011_2013',
           'Trust in local police 2015',
           'Rate maintenance of streets, sidewalks and squares 2013',
           'Rate maintenance of streets, sidewalks and squares 2015',
           'Availability of social communities and events',
           'Safety walking in neighborhood at night 2013',
           'Safety walking in community at night 2015',
           'Rate beauty or physical setting of neighborhood 2015',
           'Satisfaction with appearance of parks and squares 2013',
           'Satisfaction with local parks and squares',
           'Gender', 'Gender 2011', 'Age',
           'marital status 2011',
           'Household languages (non-english) 2015',
           'Ethnicity 2011_2013',
           'Is Hispanic 2013',
           'Ethnicity 2015',
           'Lives with Minors',
           'Housing Status',
           'Plans to leave Somerville within 2 years',
           'Years Lived in Somerville',
           'Annual Household Income',
           'Neighborhood', 'Is Student', 'Ward',
           'Precinct'
    ]

def normalize_income(income):
    if income == 'Less than $10,000':
        return income
    
    if income in ['$25,000 to $49,999', '40,000 - $49,999',
                  '$10,000 to $24,999', '30,000 - $39,999',
                  '20,000 - $29,999', '10,000 - $19,999'
                  ]:
        return '<50k'
    elif income in ['50,000 - $59,999', '60,000 - $69,999',
                   '70,000 - $79,999', '80,000 - $89,999',
                   '90,000 - $99,999', '$50,000 to $74,999',
                   '$75,000 to $99,999']:
        return '50k-100k'
    else:
        return '>100k'
    
def norm_eth(eth):
    ethnicity = eth.split(',')[0].split('/')[0].strip()
    if 'white' in ethnicity or 'asian' in ethnicity:
        if 'black' in ethnicity or 'African-American' in ethnicity:
            return 'black'
        return 'white/asian'
    elif 'Black' in ethnicity or 'African-American' in ethnicity:
        return 'Black'
    elif 'R' == ethnicity:
        return None
    elif 'American Indian' == ethnicity:
        return 'Native American'
    else:
        return ethnicity

    

happy['Income Bracket'] = happy['Annual Household Income'].apply(normalize_income)
happy['ETHNICITY'] = (happy[['Ethnicity 2011_2013', 'Ethnicity 2015']]
                      .fillna('')
                      .sum(axis=1)
                      .apply(norm_eth))



def get_income_data():
    incomes = pd.DataFrame(happy[happy['Year'] == 2011]['Income Bracket'].value_counts())
    incomes['2015'] = happy[happy['Year'] == 2015]['Income Bracket'].value_counts().sort_index()
    incomes.columns = ['2011', '2015']
    incomes['2011 %'] = incomes['2011'].apply(lambda x: x/float(incomes['2011'].sum()))
    incomes['2015 %'] = incomes['2015'].apply(lambda x: x/float(incomes['2015'].sum()))
    return incomes

def get_happy_data(year, income_bracket):
    # happiness data
    # happy[(happy['Year'] == 2015) & (happy['Income Bracket'] == '<50k')]['Current Happy'].mean()
    current_happy = (happy
        .where((happy['Year'] == year) & (happy['Income Bracket'] == income_bracket))
        ['Overall Life Satisfaction']
        .value_counts()
        .sort_index()
        .reset_index())
    current_happy['index'] = pd.to_numeric(current_happy['index'], 
                                           errors='coerce')
    current_happy['Overall Life Satisfaction'] = pd.to_numeric(current_happy['Overall Life Satisfaction'], 
                                           errors='coerce')

    current_happy = (current_happy
     .dropna()
     .where(current_happy['index'] < 11)
     .dropna(how='any')
     .sort_index())

    current_happy['Less than 6'] = current_happy['index'].apply(lambda x: x < 6)
    current_happy.groupby('Less than 6').sum()
    
writer = pd.ExcelWriter('happy_cleaned.xls')
happy.to_excel(writer, 'Sheet1')
get_income_data().to_excel(writer, 'Sheet2')
writer.save()
    




In [93]:
def get_age_bracket(age):
    if age < 13 or age > 80:
        return None
    if age < 18:
        return '<18'
    if age < 30:
        return '18 - 30'
    
    return ' 30+'
        

fio = pd.read_csv('bpd_fio.csv')

# init columns
fio['FRISKED'] = fio['OUTCOME'].astype(str).apply(lambda x: x.find('F') != -1)
fio['SEIZED'] = fio['OUTCOME'].astype(str).apply(lambda x: x.find('S') != -1)
fio['AGE_BRACKET'] = fio['AGE_AT_FIO_CORRECTED'].apply(get_age_bracket)
fio['YEAR'] = fio['FIO_DATE'].apply(lambda x: x[6:10])
fio['MONTH'] = fio['FIO_DATE'].apply(lambda x: x[0:2])
fio['DAY'] = fio['FIO_DATE'].apply(lambda x: x[3:5])

#total stop breakdown
fio['CITY'].value_counts()
# % black
fio.RACE_DESC.value_counts() / len(fio.index)

# % black by neighborhood
(fio.groupby(['CITY', 'RACE_DESC']).size() / fio.groupby(['CITY']).size()).sort_values(ascending=False)

# # city-wide makeup w/ no priors
# no_priors = fio[fio['PRIORS'] == 'NO']
# no_priors.RACE_DESC.value_counts() / len(no_priors.index)
# # neighborhood dist with no priors
# (no_priors.groupby(['CITY', 'RACE_DESC']).size() / no_priors.groupby(['CITY']).size())
# # city dist w/ priors
# has_priors = fio[fio['PRIORS'] == 'YES']
# has_priors.RACE_DESC.value_counts() / len(has_priors.index)
# # stop reasons
# (fio.groupby(['RACE_DESC', 'BASIS']).size() / fio.groupby('RACE_DESC').size())
# # terrorism
# t = fio[fio.TERRORISM == 'YES']
# t.groupby(['RACE_DESC', 'STOP_REASONS']).size()
# # % w/ priors by race *** black people that were stopped more likely to have priors
# (fio.groupby(['RACE_DESC', 'PRIORS']).size() / fio.groupby('RACE_DESC').size())
# # age distribution - median age is 25
# fio.AGE_AT_FIO_CORRECTED.median()
# # median age by race - blacks have lowest median age (24) & whites have the highest (31)
# fio[(~fio['AGE_BRACKET'].isnull()) & (fio['YEAR'] == '2015')].groupby('RACE_DESC').agg('median')['AGE_AT_FIO_CORRECTED']

CITY             RACE_DESC                           
North End        W(White)                                0.826923
Mattapan         B(Black)                                0.821826
Roxbury          B(Black)                                0.756146
Dorchester       B(Black)                                0.703664
NO DATA ENTERED  B(Black)                                0.697264
South Boston     W(White)                                0.697250
Beacon Hill      W(White)                                0.666667
West Roxbury     W(White)                                0.662437
Charlestown      W(White)                                0.638326
Hyde Park        B(Black)                                0.625432
OTHER            W(White)                                0.537283
Fenway Kenmore   B(Black)                                0.500000
Chinatown        B(Black)                                0.500000
Back Bay         W(White)                                0.486486
Allston          W(Whi

In [25]:
crime_data = pd.read_csv('crime_july2012-august2015.csv', dtype='unicode')
crime_data['REPTDISTRICT']

with open('site/public/json/neighborhoods.json') as f:
    js = json.load(f)

polygons = [shape(feature['geometry']) for feature in js['features']]
def get_neighborhood(x):
    lat, lon, count = x[0], x[1], x[2]
    if count % 500 == 0:
        print(count)
    point = Point(lon, lat)
        # check each polygon to see if it contains the point
    for i, polygon in enumerate(polygons):
        if polygon.contains(point):
            return js['features'][i]['properties']['Name']
        
    return None
            
# crime_data_2015['Neighborhood'] = crime_data_2015['Location'].apply(get_neighborhood)
crime_data_2015['location_lat'] = crime_data_2015['Location'].apply(lambda x: float(x.split(',')[0][1:]))
crime_data_2015['location_lon'] = crime_data_2015['Location'].apply(lambda x: float(x.split(',')[-1][:-1]))
crime_data_2015['count'] = crime_data_2015.reset_index().index
crime_data_2015['Neighborhood'] = crime_data_2015[['location_lat', 'location_lon', 'count']].apply(get_neighborhood, axis=1)




    

0          D4
1          B2
2          D4
3          B2
4         E18
5         C11
6          B2
7          A1
8         D14
9         E18
10         D4
11        C11
12         A1
13         D4
14         A1
15        D14
16         A1
17         C6
18         B2
19         D4
20         B2
21         B2
22        E13
23         B2
24        D14
25        E18
26         D4
27         B3
28        C11
29        E18
         ... 
268026     B2
268027    NaN
268028    E13
268029    C11
268030     B2
268031    C11
268032     B3
268033    E18
268034     D4
268035    C11
268036     B2
268037     A1
268038     A1
268039    D14
268040     C6
268041    A15
268042     C6
268043     D4
268044    NaN
268045     B2
268046    C11
268047     A1
268048     B3
268049     B3
268050     A1
268051     B3
268052    C11
268053     B2
268054     D4
268055    NaN
Name: REPTDISTRICT, dtype: object

In [18]:
crime_data_2015.to_csv('crime_data_2015.csv', sep='@')

In [10]:
crime_data_2015 = pd.read_csv('crime_data_2015.csv', sep='@')
crime_data_2015['violent'] = crime_data_2015.WEAPONTYPE.apply(lambda x: False if x in ['Unarmed', 'None'] else True)
violent_crimes = pd.DataFrame(crime_data_2015[crime_data_2015['violent'] == True].Neighborhood.value_counts()).reset_index()

violent_crimes.columns = ['Neighborhood', 'Violent Crimes']
violent_crimes = violent_crimes.set_index('Neighborhood').sort_index()

crime_data_2015[crime_data_2015["Neighborhood"] == "Back Bay"]




,Unnamed: 0,INCIDENT_TYPE_DESCRIPTION,FROMDATE,WEAPONTYPE,Shooting,DOMESTIC,Year,Month,DAY_WEEK,Location,location_lat,location_lon,count,Neighborhood,violent
2,218299,PhoneCalls,01/01/2015 12:00:00 AM,Unarmed,No,No,2015,1,Thursday,"(42.35370635, -71.07546956)",42.353706,-71.075470,2,Back Bay,False
29,218330,OTHER LARCENY,01/01/2015 12:30:00 AM,Unarmed,No,No,2015,1,Thursday,"(42.34865634, -71.08256955)",42.348656,-71.082570,29,Back Bay,False
75,218381,SIMPLE ASSAULT,01/01/2015 02:50:00 AM,Unarmed,No,No,2015,1,Thursday,"(42.34638135, -71.08482955)",42.346381,-71.084830,75,Back Bay,False
94,218402,AGGRAVATED ASSAULT,01/01/2015 04:00:00 AM,Knife,No,No,2015,1,Thursday,"(42.35456135, -71.07570456)",42.354561,-71.075705,94,Back Bay,True
126,218441,OTHER LARCENY,01/01/2015 09:45:00 AM,Unarmed,No,No,2015,1,Thursday,"(42.35096134, -71.07410456)",42.350961,-71.074105,126,Back Bay,False
137,218457,OTHER LARCENY,01/01/2015 11:00:00 AM,Unarmed,No,No,2015,1,Thursday,"(42.34710135, -71.07960455)",42.347101,-71.079605,137,Back Bay,False
145,218468,InvPer,01/01/2015 12:00:00 PM,Other,No,No,2015,1,Thursday,"(42.35058135, -71.07548456)",42.350581,-71.075485,145,Back Bay,True
181,218515,OTHER LARCENY,01/01/2015 03:20:00 PM,Unarmed,No,No,2015,1,Thursday,"(42.34904635, -71.08104955)",42.349046,-71.081050,181,Back Bay,False
201,218538,VAL,01/01/2015 05:54:00 PM,Unarmed,No,No,2015,1,Thursday,"(42.34851634, -71.08311455)",42.348516,-71.083115,201,Back Bay,False
202,218539,OTHER LARCENY,01/01/2015 05:58:00 PM,Unarmed,No,No,2015,1,Thursday,"(42.34710135, -71.07960455)",42.347101,-71.079605,202,Back Bay,False


In [12]:
census = pd.read_csv('ma_census_2015.csv', dtype='unicode', skiprows=1)
tract_data = pd.read_csv('tract_data.csv', delimiter=' ')
pop = pd.read_csv('pop_total.csv', dtype='unicode', skiprows=1)



def get_tract_id(label):
    s = label.split(',')
    return s[0][12:]

def wavg(group, avg_name, weight_name):
    """ 
    http://stackoverflow.com/questions/10951341/pandas-dataframe-aggregate-function-using-multiple-columns
    """
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return d.mean()

    
census['tract'] = census['Geography'].apply(get_tract_id)
census = (census
          .rename(columns = {'Id2':'GEOID'})
          .set_index('GEOID'))
pop = (pop
          .rename(columns = {'Id2':'GEOID'})
          .set_index('GEOID'))
pop = pd.DataFrame(pop['Estimate; Total'])
tract_data['GEOID'] = tract_data['GEOID'].astype(str)
tract_data = tract_data.set_index('GEOID')

census = census.join(tract_data, how='inner').join(pop, how='inner')

census = census[['Neighborhood', 'tract','Estimate; Total', 'Percent; EMPLOYMENT STATUS - Population 16 years and over - In labor force - Civilian labor force - Unemployed', 'Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All families']]
census.columns = ['Neighborhood', 'tract', 'pop', 'unemployment rate', 'percent families below poverty level']
census = census.replace('-', np.nan)
census[['pop', 'unemployment rate', 'percent families below poverty level']] = census[['pop', 'unemployment rate', 'percent families below poverty level']].astype(float)

census[census['Neighborhood'] == 'North_End']

employment_by_neighborhood = census.groupby('Neighborhood').apply(wavg, 'unemployment rate', 'pop')
poverty_by_neighborhood = census.groupby('Neighborhood').apply(wavg, 'percent families below poverty level', 'pop')

avgs = pd.DataFrame(employment_by_neighborhood, columns=['unemploymentPct'])
avgs['povertyPct'] = poverty_by_neighborhood
avgs['pop'] = census.groupby('Neighborhood').sum()['pop']
avgs = avgs.reset_index()
avgs['Neighborhood'] = avgs['Neighborhood'].apply(lambda x: x.replace('_', ' '))
avgs = avgs.set_index('Neighborhood').sort_index()

avgs = avgs.join(violent_crimes, how='inner')
avgs['violentCrimePerCapita'] = avgs['Violent Crimes'] / avgs['pop']
avgs.to_csv('site/public/csv/averages_2015.csv')
## add
## 1) population
## 2) violent crimes
## 3) violent crimes per capita
